In [1]:
print("Hello World")

Hello World


In [4]:
!pip install kafka-python
!pip install pandas
!pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     -------------------------------------- - 2.9/3.0 MB 21.0 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 17.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 18.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.18.1-py3-none-any.whl size=139100 sha256=d1835cbb34f746bf6ab86871897ec2655aa7620b000812e734e7fa4e945a5791
  Stored in directory: c:\users\arulk\appdata\local\pip\cache\wheels\25\cb\79\a133a0d1d75f318a96614ed7fb97bdf2f35a7b6c4d4e426e3f
Successfully b

In [1]:
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
from time import sleep
from json import dumps
import json
import time

In [2]:
producer = KafkaProducer(bootstrap_servers=['43.205.231.111:9092'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [3]:
for i in range(5):
    message = {"message": "Hello World"}
    producer.send('demo_testing1', value=message)
    print(f"Sent: {message}")
    time.sleep(1)  # Optional: wait a bit between messages

producer.flush()  

Sent: {'message': 'Hello World'}
Sent: {'message': 'Hello World'}
Sent: {'message': 'Hello World'}
Sent: {'message': 'Hello World'}
Sent: {'message': 'Hello World'}


In [4]:
df = pd.read_csv(r'D:\Python_Projects\DATA_ENGINEERING\kafka-stock-market-project-key\indexProcessed.csv')

In [5]:
df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [6]:
while True:
    dict_stock=df.sample(1).to_dict(orient="records")[0]  #data frame to dictionary format
    producer.send('demo_testing1', value=dict_stock)

KeyboardInterrupt: 

In [1]:
#final code
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

# Create Kafka producer
producer = KafkaProducer(
    bootstrap_servers=['13.233.92.65:9092'],  # Replace with your broker IP
    value_serializer=lambda x: dumps(x).encode('utf-8')  # Serialize dict to JSON bytes
)

# Load dataset
df = pd.read_csv(r'D:\Python_Projects\DATA_ENGINEERING\kafka-stock-market-project-key\indexProcessed.csv')

# Send data to Kafka topic continuously
while True:
    # Pick one random row from dataframe
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    
    # Send the row to Kafka topic
    producer.send('demo_testing1', value=dict_stock)
    producer.flush()  # Ensure the message is sent immediately
    
    print(f"Sent: {dict_stock}")  # Optional logging
    sleep(1)  # Add a small delay to simulate real-time data


Sent: {'Index': 'N225', 'Date': '1989-06-09', 'Open': 33755.69141, 'High': 33812.23047, 'Low': 33581.48828, 'Close': 33639.98047, 'Adj Close': 33639.98047, 'Volume': 0.0, 'CloseUSD': 336.3998047}
Sent: {'Index': 'TWII', 'Date': '2021-03-10', 'Open': 15921.4502, 'High': 15986.07031, 'Low': 15857.32031, 'Close': 15911.66992, 'Adj Close': 15911.66992, 'Volume': 4340600.0, 'CloseUSD': 636.4667968}
Sent: {'Index': 'N225', 'Date': '1994-03-11', 'Open': 20109.91016, 'High': 20256.23047, 'Low': 20017.09961, 'Close': 20115.31055, 'Adj Close': 20115.31055, 'Volume': 0.0, 'CloseUSD': 201.1531055}
Sent: {'Index': 'SSMI', 'Date': '1995-06-08', 'Open': 2819.899902, 'High': 2832.800049, 'Low': 2801.399902, 'Close': 2806.800049, 'Adj Close': 2806.800049, 'Volume': 0.0, 'CloseUSD': 3115.5480543900003}
Sent: {'Index': 'IXIC', 'Date': '2013-03-12', 'Open': 3244.850098, 'High': 3249.780029, 'Low': 3229.919922, 'Close': 3242.320068, 'Adj Close': 3242.320068, 'Volume': 1673740000.0, 'CloseUSD': 3242.320068}

KeyboardInterrupt: 

In [7]:
import yfinance as yf
import pandas as pd
from kafka import KafkaProducer
from json import dumps
from time import sleep

# === Step 1: Fetch Historical Stock Data ===
data = yf.download('^HSI', start='1986-12-31', end='1987-01-05', auto_adjust=False)

# Reset index to make 'Date' a column
data.reset_index(inplace=True)
data['Index'] = 'HSI'

# Fill missing values
data['Volume'] = data['Volume'].fillna(0).astype(int)
data['Adj Close'] = data['Adj Close'].fillna(data['Close'])

# Compute CloseUSD (e.g., 1 HKD = 0.13 USD)
data['CloseUSD'] = data['Close'] * 0.13

# Format Date
data['Date'] = data['Date'].dt.strftime('%d-%m-%Y')

# Reorder columns
data = data[['Index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CloseUSD']]

# Flatten column names to avoid tuple key issues
data.columns = data.columns.map(str)

# === Step 2: Kafka Producer ===
producer = KafkaProducer(
    bootstrap_servers=['13.233.92.65:9092'],
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

# === Step 3: Send to Kafka Topic ===
for _, row in data.iterrows():
    message = row.to_dict()
    producer.send('demo_testing1', value=message)
    producer.flush()
    print(f"Sent: {message}")
    sleep(1)

print("✅ All records sent to Kafka.")


[*********************100%***********************]  1 of 1 completed


Sent: {"('Index', '')": 'HSI', "('Date', '')": '31-12-1986', "('Open', '^HSI')": 2568.300048828125, "('High', '^HSI')": 2568.300048828125, "('Low', '^HSI')": 2568.300048828125, "('Close', '^HSI')": 2568.300048828125, "('Adj Close', '^HSI')": 2568.300048828125, "('Volume', '^HSI')": 0, "('CloseUSD', '')": 333.8790063476563}
Sent: {"('Index', '')": 'HSI', "('Date', '')": '02-01-1987', "('Open', '^HSI')": 2540.10009765625, "('High', '^HSI')": 2540.10009765625, "('Low', '^HSI')": 2540.10009765625, "('Close', '^HSI')": 2540.10009765625, "('Adj Close', '^HSI')": 2540.10009765625, "('Volume', '^HSI')": 0, "('CloseUSD', '')": 330.2130126953125}
✅ All records sent to Kafka.


In [9]:
import yfinance as yf
import pandas as pd
from kafka import KafkaProducer
from json import dumps
from time import sleep

# === Step 1: Download Data ===
ticker = '^HSI'
data = yf.download(ticker, start='1986-12-31', end='1987-01-05', auto_adjust=False)

# === Step 2: Reset Index to make 'Date' a column ===
data.reset_index(inplace=True)
data['Index'] = 'HSI'

# === Step 3: Flatten MultiIndex Columns ===
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]

# === Step 4: Clean and Arrange Columns ===
data['Volume'] = data['Volume'].fillna(0).astype(int)
data['Adj Close'] = data['Adj Close'].fillna(data['Close'])
data['CloseUSD'] = data['Close'] * 0.13
data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%d-%m-%Y')

# Final column order
data = data[['Index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CloseUSD']]

# === Step 5: Kafka Producer ===
producer = KafkaProducer(
    bootstrap_servers=['13.233.92.65:9092'],  # Your Kafka broker IP
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

# === Step 6: Send Each Record to Kafka ===
for _, row in data.iterrows():
    message = row.to_dict()
    producer.send('demo_testing1', value=message)
    producer.flush()
    print(f"Sent: {message}")
    sleep(1)


[*********************100%***********************]  1 of 1 completed


Sent: {'Index': 'HSI', 'Date': '31-12-1986', 'Open': 2568.300048828125, 'High': 2568.300048828125, 'Low': 2568.300048828125, 'Close': 2568.300048828125, 'Adj Close': 2568.300048828125, 'Volume': 0, 'CloseUSD': 333.8790063476563}
Sent: {'Index': 'HSI', 'Date': '02-01-1987', 'Open': 2540.10009765625, 'High': 2540.10009765625, 'Low': 2540.10009765625, 'Close': 2540.10009765625, 'Adj Close': 2540.10009765625, 'Volume': 0, 'CloseUSD': 330.2130126953125}


In [20]:
import yfinance as yf
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps

# Step 1: Fetch historical stock data
ticker = '^HSI'
data = yf.download(ticker, start='1986-12-31', end='1987-01-05', auto_adjust=False)

# Step 2: Flatten MultiIndex columns if they exist
data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]

# Step 3: Format and enrich data
data.reset_index(inplace=True)
data['Date'] = data['Date'].dt.strftime('%d-%m-%Y')
data['Index'] = 'HSI'
data['CloseUSD'] = data['Close'] * 0.13
data['Volume'] = data['Volume'].fillna(0).astype(int)

# Reorder columns
data = data[['Index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CloseUSD']]

# Step 4: Create Kafka producer
producer = KafkaProducer(
    bootstrap_servers=['13.233.92.65:9092'],  # Replace with your Kafka IP
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

# Step 5: Stream data to Kafka until manually stopped
print("✅ Kafka Producer started. Streaming data... Press Ctrl+C to stop.")
try:
    while True:
        for _, row in data.iterrows():
            producer.send('demo_testing1', value=row.to_dict())
            producer.flush()
            print(f"Sent: {row.to_dict()}")
            sleep(1)
except KeyboardInterrupt:
    print("🛑 Stopped by user.")
finally:
    producer.close()


[*********************100%***********************]  1 of 1 completed


✅ Kafka Producer started. Streaming data... Press Ctrl+C to stop.
Sent: {'Index': 'HSI', 'Date': '31-12-1986', 'Open': 2568.300048828125, 'High': 2568.300048828125, 'Low': 2568.300048828125, 'Close': 2568.300048828125, 'Adj Close': 2568.300048828125, 'Volume': 0, 'CloseUSD': 333.8790063476563}
Sent: {'Index': 'HSI', 'Date': '02-01-1987', 'Open': 2540.10009765625, 'High': 2540.10009765625, 'Low': 2540.10009765625, 'Close': 2540.10009765625, 'Adj Close': 2540.10009765625, 'Volume': 0, 'CloseUSD': 330.2130126953125}
Sent: {'Index': 'HSI', 'Date': '31-12-1986', 'Open': 2568.300048828125, 'High': 2568.300048828125, 'Low': 2568.300048828125, 'Close': 2568.300048828125, 'Adj Close': 2568.300048828125, 'Volume': 0, 'CloseUSD': 333.8790063476563}
Sent: {'Index': 'HSI', 'Date': '02-01-1987', 'Open': 2540.10009765625, 'High': 2540.10009765625, 'Low': 2540.10009765625, 'Close': 2540.10009765625, 'Adj Close': 2540.10009765625, 'Volume': 0, 'CloseUSD': 330.2130126953125}
Sent: {'Index': 'HSI', 'Date